## **Importing all the modules**


YOLO (from ultralytics): load and run the YOLOv8 model.

cv2: read/write images, draw boxes, color conversions.

numpy as np: image arrays and numeric operations.

typing.List/Dict: type annotations for clarity.

torch: detect GPU availability (torch.cuda.is_available()).

os: read env vars and check file paths.

In [1]:
from ultralytics import YOLO
import cv2
import numpy as np
from typing import List, Dict
import torch


Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\HP\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [25]:
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path="D:\PycharmProjects\Eco_Vision\Backend\.env")

<>:3: SyntaxWarning: invalid escape sequence '\P'
<>:3: SyntaxWarning: invalid escape sequence '\P'
C:\Users\HP\AppData\Local\Temp\ipykernel_7424\2373333911.py:3: SyntaxWarning: invalid escape sequence '\P'
  load_dotenv(dotenv_path="D:\PycharmProjects\Eco_Vision\Backend\.env")


True

_**device:**_ force CPU/GPU manually if you want otherwise we auto-detect later.
Pick **_model_path_** use environment variable MODEL_PATH, else default to "yolov8n.pt".
_**YOLO(model_path):**_calls the YOLO class (from Ultralytics) with model_path.
         -> This loads the pretrained model into memory.
         -> **NOTE:** we load the model once in the constructor when I write the script, so we don’t have to reload it every time we detect something.


In [27]:
model_path = os.getenv("MODEL_PATH",None) #if we didn't set model , default this come "yolov8n.pt"
device = ("cuda" if torch.cuda.is_available() else "cpu")
model = YOLO(model_path)

FileNotFoundError: [Errno 2] No such file or directory: 'yolov12n.pt'

#### as here  below we can see our model is lock & loaded on cpu not gpu as i dont have didicated graphic card 💀.

In [7]:
print(f"[YOLODetector] Loaded model: {model_path} on {device}")

[YOLODetector] Loaded model: yolov8n.pt on cpu


we are using very small model here and it it fatest